In [2]:
from glob import glob
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet
import shutil

In [3]:
path = glob('./data/number-plate-detection/images/*.xml')
def get_label_xml(path):
    labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
    for filename in path:

        info = xet.parse(filename)
        root = info.getroot()
        member_object = root.find('object')
        labels_info = member_object.find('bndbox')
        xmin = int(labels_info.find('xmin').text)
        xmax = int(labels_info.find('xmax').text)
        ymin = int(labels_info.find('ymin').text)
        ymax = int(labels_info.find('ymax').text)

        labels_dict['filepath'].append(filename)
        labels_dict['xmin'].append(xmin)
        labels_dict['xmax'].append(xmax)
        labels_dict['ymin'].append(ymin)
        labels_dict['ymax'].append(ymax)
    return labels_dict
labels_dict = get_label_xml(path)
labels_dict = pd.DataFrame(labels_dict)

In [4]:
def getFilename(filename,str='./data/number-plate-detection/images'):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join(str,filename_image)
    return filepath_image


filename1 = labels_dict['filepath'][0]
filename1 = getFilename(filename1)
filename1 = labels_dict['filepath'].apply(getFilename)
print(filename1.shape)

(225,)


In [5]:
df = labels_dict

path_2 = glob('./data/(2)/annotations/*.xml')
labels_dict = get_label_xml(path_2)
labels_dict = pd.DataFrame(labels_dict)

filename2 = labels_dict['filepath'][0]
filename2 = getFilename(filename2,str='./data/(2)/images')
filename2 = labels_dict['filepath'].apply(getFilename,str='./data/(2)/images')

In [6]:
df = pd.concat([df,labels_dict],ignore_index=True)
filename = pd.concat([filename1,filename2],ignore_index=True)
print(df.shape)
print(filename.shape)
image_path = list(filename)

(658, 5)
(658,)


In [7]:
path_train_images = './data_plate_detect/data/train/images/'
path_train_labels = './data_plate_detect/data/train/labels/'
path_val_images = './data_plate_detect/data/val/images/'
path_val_labels = './data_plate_detect/data/val/labels/'
def check_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)
        print("Created folder: ", path)
    else:
        shutil.rmtree(path)
        os.makedirs(path)
        print("Already exists: ", path)

check_folder(path_train_images)
check_folder(path_train_labels)
check_folder(path_val_images)
check_folder(path_val_labels)

Already exists:  ./data_plate_detect/data/train/images/
Already exists:  ./data_plate_detect/data/train/labels/
Already exists:  ./data_plate_detect/data/val/images/
Already exists:  ./data_plate_detect/data/val/labels/


In [8]:
cnt = 0
for i in range(len(image_path)):
    imageID = image_path[i].split('/')[-1].split('.')[0]
    imageID = imageID.split('\\')[-1]
    # print(imageID)
    x1, y1, x2, y2 = df['xmin'][i], df['ymin'][i], df['xmax'][i], df['ymax'][i]
    x,y,w,h = x1, y1, x2-x1, y2-y1
    image = cv2.imread(image_path[i])
    x = x / image.shape[1]
    y = y / image.shape[0]
    w = w / image.shape[1]
    h = h / image.shape[0]
    cnt+=1
    path_images = path_val_images
    path_labels = path_val_labels
    if cnt<=0.9*len(image_path):
        path_images = path_train_images
        path_labels = path_train_labels
    cv2.imwrite(path_images + imageID + '.jpg', image)
    if os.path.exists(path_labels + imageID + '.txt') != True:
        with open(path_labels + imageID + '.txt', 'w') as f:
            f.write(str(0) + ' ' + str(x) + ' ' + str(y) + ' ' + str(w) + ' ' + str(h))
    else:
        with open(path_labels + imageID + '.txt', 'a') as f:
            f.write('\n' + str(0) + ' ' + str(x) + ' ' + str(y) + ' ' + str(w) + ' ' + str(h))